In [32]:
import gensim as gs
from nltk.tokenize import WordPunctTokenizer
import networkx as nx
from gensim import models

w2v = models.Word2Vec.load_word2vec_format(
            'ruscorpora_mean_hs.model.bin'
            , binary=True
            , encoding='utf-8'
        )

In [47]:
def delete_tag(word):
    return word[ : len(word) - word[::-1].find('_') - 1 ]

In [64]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def inflect(adjective, noun):
    try:
        p = morph.parse(noun)
        gender = p[0].tag.gender
        number = p[0].tag.number
        form = set()

        if gender is not None:
            form.add(gender)

        if number is not None:
            form.add(number)

        p_adj = morph.parse(adjective)[0]
        
        return p_adj.inflect(form)[0]

    except BaseException:
        return adjective

In [39]:
maked_word = 'язык' + '_S'

text = open('trt_stemmed.txt', 'r').read()

tokens = WordPunctTokenizer().tokenize(text)

def part_of_speech_filter(word):
    tag_pos = word.find('_') + 1
    if tag_pos == 0:
        return False
    tag = word[tag_pos:]
    return tag == 'S' or tag == 'A' or tag == 'ANUM' or tag == 'APRO' or tag == 'ADV'

unique_words = list(filter(part_of_speech_filter, set(tokens)))

In [40]:
gr = nx.Graph()
gr.add_nodes_from(unique_words)

unique_words_set = set(unique_words)

max_weight = 0
for i in range(len(tokens) - 2):
    if not tokens[i] in unique_words_set:
        continue
    for j in range(1, 3):
        edge_data = gr.get_edge_data(tokens[i], tokens[i+j])
        weight = 1./j if edge_data is None else edge_data['weight'] + 1./j
        gr.add_edge(tokens[i], tokens[i+j], weight=weight)
        if max_weight < weight:
            max_weight = weight

In [41]:
calculated_page_rank = nx.pagerank(gr, weight='weight')

print(calculated_page_rank)

for word in tokens:
    print((' ! ' if word in calculated_page_rank and calculated_page_rank[word] > 0.004 else '   ') + word)

{'носитель_S': 0.003020947680708345, 'человекообразный_A': 0.0004792517284229617, 'выделять_V': 0.0005154202052747533, 'таксон_S': 0.0004983368691926454, 'психолог_S': 0.0009727407075557102, 'пример_S': 0.0006436914373029506, 'такой_APRO': 0.001481737494870013, 'англ_S': 0.0006614717819247388, 'довольно_ADV': 0.00047654218422525326, 'малый_A': 0.0005452168487328997, 'свой_APRO': 0.0016967001251224456, 'неандерталец_S': 0.00046009398048811525, 'профессиональный_A': 0.0005042802007783611, 'литература_S': 0.0016355541075371489, 'олонецкий_A': 0.0006704266621492652, 'глобализация_S': 0.0004848806706868831, 'благополучный_A': 0.0004127019786114635, 'независимый_A': 0.0004643299585724874, 'художественный_A': 0.0007470585353379626, 'примитивный_A': 0.0006782791504313414, 'следующий_A': 0.0010163505389423617, 'распределение_S': 0.0006617597134664695, 'тыс_S': 0.0007498980385353169, 'территориальный_A': 0.001062151750556295, 'глоттогония_S': 0.0005298943097396235, 'настаивать_V': 0.000322631005

In [42]:
for first in unique_words:
    for second in unique_words:
        try:
            similarity=w2v.similarity(first, second)
            
            edge_data = gr.get_edge_data(first, second)
            weight = similarity if edge_data is None else edge_data['weight'] / max_weight + similarity
            
            if weight > 0.4:
                gr.add_edge(first, second, weight=weight)
        except BaseException as e:
            pass

In [43]:
calculated_page_rank = nx.pagerank(gr, weight='weight')

print(calculated_page_rank)

for word in tokens:
    print((' ! ' if word in calculated_page_rank and calculated_page_rank[word] > 0.004 else '   ') + word)

{'носитель_S': 0.0027248639836397765, 'человекообразный_A': 0.00052856083203184454, 'выделять_V': 0.000449253251039464, 'таксон_S': 0.00058888263882345268, 'психолог_S': 0.0012720546858401374, 'пример_S': 0.00069662144148644476, 'такой_APRO': 0.0011817203907660834, 'англ_S': 0.00076604562026404773, 'довольно_ADV': 0.0011935018866524438, 'малый_A': 0.00066588788619383434, 'свой_APRO': 0.0014026322144070741, 'неандерталец_S': 0.00055583722603170897, 'профессиональный_A': 0.00060190742509103263, 'литература_S': 0.0020741586598879334, 'олонецкий_A': 0.00070980577459086409, 'глобализация_S': 0.00062134212637214212, 'благополучный_A': 0.00065153275780688432, 'независимый_A': 0.00060056436585346801, 'художественный_A': 0.00096951457396651775, 'примитивный_A': 0.00099778510897827362, 'следующий_A': 0.001289026510336487, 'распределение_S': 0.00076042244401571186, 'тыс_S': 0.00090320036185101643, 'территориальный_A': 0.0010271157718162718, 'глоттогония_S': 0.00044934059228997364, 'настаивать_V':

In [65]:
max_id = -1
max_prob = 0

for i in range(len(tokens) - 1):
    if not tokens[i] in unique_words_set or not tokens[i+1] in unique_words_set:
        continue
        
    if tokens[i] == maked_word or tokens[i+1] == maked_word:
        continue
    
    prob = calculated_page_rank[tokens[i]] * calculated_page_rank[tokens[i+1]]
    
    if prob > 0.00001:
        if tokens[i][-2:] == '_A' and tokens[i+1][-2:] == '_S':
            print(inflect(delete_tag(tokens[i]), delete_tag(tokens[i+1])) + ' ' + delete_tag(tokens[i+1]))
        else:
            print(delete_tag(tokens[i]) + ' ' + delete_tag(tokens[i+1]))
    
    if max_prob < prob:
        max_prob = prob
        max_id = i

print(tokens[max_id] + ' ' + tokens[max_id+1])

знаковая система
знаковая система
знаковая система
знаковая система
изучение лингвистика
общение человек
человек например
знаковая система
например система
знаковая система
система предмет
другой функция
весь функция
функция общение
основная функция
функция функция
знаковая система
функция использование
текст основной
система общение
человек другой
другой теория
явление который
система общение
функция человек
другой теория
речь теория
этот теория
другой исследователь
этот теория
система коммуникация
этот система
система коммуникация
 уже
назад другой
назад человек
 другой
языковая семья
языковая информация
историческая лингвистика
 лингвистический
элемент языковой
языковая система
языковая единица
текст основной
статья языковой
языковая система
система уровень
весь система
языковая система
текст основной
мир языковой
языковая семья
языковая семья
 классификация
текст основной
определённая система
система письменность
языковой ресурс
который уже
текст основной
основная разновидность
диа